In [1]:
import pandas as pd
import numpy as np
from collections import Counter
#import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

# The end of this workflow is different from the original scHeart.ipynb notebook located in /Users/stearb/Dropbox/CHOP/R03/code/scHeart, we are using Jonathan Silversteins workflow for the Neo4j CSV creation (meaning the files produced by this workflow will be the inputs into JS's workflow) ...so we only need to create 2 files, a nodes.tsv and an edges.tsv (instead of the ~6 files, CUIs, CUI-CUIs, Code-CUIs, Terms, etc.)

## The guide for how to create these new nodes and edges files can be found in the Data Distillerys [github](https://github.com/dbmi-pitt/UBKG/tree/main/user%20guide)

In [13]:
#umls_dir = '/Users/stearb/Desktop/hubmap-kg/new_build_csv_data/'
umls_dir = '/Users/stearb/Desktop/DESKTOP_TRANSFER/DataDistilleryDistributions/DataDistillery10Sept2023/DataDistillery10September2023/'

In [3]:
# !jupyter nbconvert --to script scHeart_JS.ipynb

In [4]:
## Flatten/melt the average gene expression data so we have 1 gene and 1 tissue per row
#df = pd.read_csv('/Users/stearb/Desktop/R03_local/data/scHeart/asp_average_gene_expression_14_celltypes.csv')
#df.rename(columns={'Unnamed: 0':'Gene'},inplace=True)
#print(df.shape)

#df2 = pd.melt(df, id_vars="Gene",value_name="Average Exp")
#df2['Average Exp'].plot(kind='hist',range=[0,10],bins=50)
#df2.to_csv('/Users/stearb/Desktop/R03_local/data/scHeart/asp_average_gene_expression_14_celltypes_LONG.csv')

In [5]:
#cui_codes_temp = pd.read_csv('/Users/stearb/Desktop/R03_local/data/scHeart/temp_files/CUI-CODEs.csv')
#cui_codes_temp

In [14]:
z = pd.read_csv('/Users/stearb/Desktop/DESKTOP_TRANSFER/R03_local/data/scHeart/asp_cell_type_markers_zeros_labelled.csv')
z.drop(['p_val','pct.1','pct.2'],axis=1,inplace=True)
print(len(z))
z.head(3)   # this data has already been filtered to remove p-vals greater than .06

2932


,avg_log2FC,p_val_adj,cluster,gene,cluster_celltype_name
0,5.513025,0.0,7,BPGM,Erythrocytes
1,5.509262,0.0,7,MT1H,Erythrocytes
2,4.710317,0.0,7,GYPB,Erythrocytes


In [15]:
#### Merge in HGNC IDs
hgnc_master = pd.read_csv('/Users/stearb/Desktop/DESKTOP_TRANSFER/R03_local/data/use_config/HELPER_FILES/hgnc_master.txt',sep='\t')
gene_map = hgnc_master[['hgnc_id','symbol']].rename(columns={'symbol':'gene'})

z2 = pd.merge(z,gene_map,on='gene')

print(len(z2))

2816


In [16]:
# We lost 116 rows when merging in the hgnc ids.

In [17]:
# Here are the rows we lost.
#z[~z['gene'].isin(gene_map['gene'])].sample(3)

In [18]:
umls_dir

'/Users/stearb/Desktop/DESKTOP_TRANSFER/DataDistilleryDistributions/DataDistillery10Sept2023/DataDistillery10September2023/'

In [20]:
#### Merge in HGNC CUIs

# First, GET CUI - HGNC CODE MAPPINGS STRAIGHT FROM CSVs
UMLS_CUI_CODEs = pd.read_csv(umls_dir+'CUI-CODEs.csv')

umls_genes = UMLS_CUI_CODEs[UMLS_CUI_CODEs[':END_ID'].str.startswith('HGNC')].rename(
                                    columns={':START_ID':'CUI_hgnc',':END_ID':'hgnc_id'})

umls_genes['hgnc_id'] = [i.split(':')[1] for i in umls_genes['hgnc_id']]

z3 = pd.merge(z2,umls_genes,on='hgnc_id')

print(len(z3))

0


In [23]:
umls_genes['hgnc_id']

7285         3513
12399         546
12404        1836
12405        1952
12415        3444
            ...  
17234239     9634
17234240    11445
17234241     4883
17234242    14007
17234243    10001
Name: hgnc_id, Length: 43178, dtype: object

In [25]:
z3 = z2

In [26]:
z3['cluster_celltype_name_NO_SPACE'] = [i.replace(' ','-') for i in z3['cluster_celltype_name'] ]
z3['cluster_celltype_name_NO_SPACE'] = [i.replace('_','-') for i in z3['cluster_celltype_name_NO_SPACE'] ]

z3['cluster_celltype_name_NO_SPACE'] = [i.replace('/','slash') for i in z3['cluster_celltype_name_NO_SPACE'] ]
z3['cluster_celltype_name_NO_SPACE'] = [i.replace('&','and') for i in z3['cluster_celltype_name_NO_SPACE'] ]

z3['hgnc_id_dashes'] =  [ i.replace(':','-') for i in z3['hgnc_id'] ]

In [27]:
z3[z3['cluster_celltype_name_NO_SPACE'].str.contains('&')]

,avg_log2FC,p_val_adj,cluster,gene,cluster_celltype_name,hgnc_id,cluster_celltype_name_NO_SPACE,hgnc_id_dashes


In [28]:
### Create the scHeart SAB
#z3['SAB'] = 'scHeart_PMID_31835037'

z3['SAB'] = 'SCHEART'

# Create scHeart CODE

# assert that the celltype and hgncid col will uniquely identify each code node
assert len(z3) == len(z3[['cluster_celltype_name_NO_SPACE','hgnc_id']].drop_duplicates()) 

z3['CODE'] = z3['cluster_celltype_name_NO_SPACE'] + '-' + z3['hgnc_id_dashes']

# Create scHeart CODEID
z3['CODEID'] = z3['SAB'] + ' ' + z3['CODE']

# Create scHeart CUIs
#z3['CUI:ID'] = 'CUI_SAB'+z3['CODEID']

#z3.drop(['cluster','gene','hgnc_id'],axis=1,inplace=True)

z3.head(3)

,avg_log2FC,p_val_adj,cluster,gene,cluster_celltype_name,hgnc_id,cluster_celltype_name_NO_SPACE,hgnc_id_dashes,SAB,CODE,CODEID
0,5.513025,0.000000,7,BPGM,Erythrocytes,HGNC:1093,Erythrocytes,HGNC-1093,SCHEART,Erythrocytes-HGNC-1093,SCHEART Erythrocytes-HGNC-1093
1,-2.470632,0.010812,6,BPGM,Epicardium-derived cells,HGNC:1093,Epicardium-derived-cells,HGNC-1093,SCHEART,Epicardium-derived-cells-HGNC-1093,SCHEART Epicardium-derived-cells-HGNC-1093
2,-1.961892,0.000016,14,BPGM,Ventricular cardiomyocytes,HGNC:1093,Ventricular-cardiomyocytes,HGNC-1093,SCHEART,Ventricular-cardiomyocytes-HGNC-1093,SCHEART Ventricular-cardiomyocytes-HGNC-1093


#### Create cell type nodes

In [29]:
# Now create the celltype nodes
z3.rename(columns={'cluster_celltype_name_NO_SPACE':'celltype_CODE'},inplace=True)

In [30]:
z3['celltype_CODE'] = [i.replace('/','slash') for i in z3['celltype_CODE'] ]
z3['celltype_CODE'] = [i.replace('&','and') for i in z3['celltype_CODE'] ]

In [31]:
z3['celltype_SAB'] = 'author_defined_cluster'
z3['celltype_CODEID'] = z3['celltype_SAB'] + ' ' + z3['celltype_CODE']
#z3['celltype_CUI'] = 'CUI_SAB ' + z3['celltype_CODEID']

In [32]:
z3[z3['celltype_CODE'].str.contains('&')]

,avg_log2FC,p_val_adj,cluster,gene,cluster_celltype_name,hgnc_id,celltype_CODE,hgnc_id_dashes,SAB,CODE,CODEID,celltype_SAB,celltype_CODEID


In [33]:
z3.head(3)

,avg_log2FC,p_val_adj,cluster,gene,cluster_celltype_name,hgnc_id,celltype_CODE,hgnc_id_dashes,SAB,CODE,CODEID,celltype_SAB,celltype_CODEID
0,5.513025,0.000000,7,BPGM,Erythrocytes,HGNC:1093,Erythrocytes,HGNC-1093,SCHEART,Erythrocytes-HGNC-1093,SCHEART Erythrocytes-HGNC-1093,author_defined_cluster,author_defined_cluster Erythrocytes
1,-2.470632,0.010812,6,BPGM,Epicardium-derived cells,HGNC:1093,Epicardium-derived-cells,HGNC-1093,SCHEART,Epicardium-derived-cells-HGNC-1093,SCHEART Epicardium-derived-cells-HGNC-1093,author_defined_cluster,author_defined_cluster Epicardium-derived-cells
2,-1.961892,0.000016,14,BPGM,Ventricular cardiomyocytes,HGNC:1093,Ventricular-cardiomyocytes,HGNC-1093,SCHEART,Ventricular-cardiomyocytes-HGNC-1093,SCHEART Ventricular-cardiomyocytes-HGNC-1093,author_defined_cluster,author_defined_cluster Ventricular-cardiomyocytes


## Load in main dataset
The data_w_bins.csv file is the most up-to-date

In [34]:
#fulldata = pd.read_csv('/Users/stearb/Desktop/R03_local/data/scHeart/data_w_bins.csv')
#fulldata['CODEID'] = fulldata['SAB'] + fulldata['preCODE']
#fulldata['CUI:ID'] = CUIbase64(fulldata['CODEID'])
#fulldata.head(3)

fulldata = z3

# Remove rows w/ p val greater than .06 

In [35]:
pre=len(fulldata)

fulldata = fulldata[fulldata['p_val_adj'] < .06]

post=len(fulldata)

print(pre-post)

0


In [23]:
fulldata['celltype_CODEID'].unique() 

array(['author_defined_cluster Erythrocytes',
       'author_defined_cluster Epicardium-derived-cells',
       'author_defined_cluster Ventricular-cardiomyocytes',
       'author_defined_cluster Smooth-muscle-cells-slash-fibroblast-like)',
       'author_defined_cluster Fibroblast-like-(related-to-cardiac-skeleton-connective-tissue)',
       'author_defined_cluster Capillary-endothelium',
       'author_defined_cluster Endothelium-slash-pericytes-slash-adventitia',
       'author_defined_cluster Immune-cells',
       'author_defined_cluster Fibroblast-like-(related-to-smaller-vascular-development)',
       'author_defined_cluster Cardiac-neural-crest-cells-and-Schwann-progenitor-cells',
       'author_defined_cluster Fibroblast-like-(related-to-larger-vascular-development)',
       'author_defined_cluster Epicardial-cells',
       'author_defined_cluster Myoz2-enriched-cardiomyocytes',
       'author_defined_cluster Atrial-cardiomyocytes'], dtype=object)

# Create edges file

In [36]:
fulldata['hgnc_codeID'] = 'HGNC '+ fulldata['hgnc_id']
fulldata['hgnc_singleCell_predicate'] = 'RO:0002206'  # 'expressed in'   ,  OLD rel: 'gene_has_single_cell_expression'

# hgnc to single cell
edges1 = fulldata[['CODEID','hgnc_singleCell_predicate','hgnc_codeID']]
edges1.columns = ['subject','predicate','object']
edges1.head(2)

,subject,predicate,object
0,SCHEART Erythrocytes-HGNC-1093,RO:0002206,HGNC HGNC:1093
1,SCHEART Epicardium-derived-cells-HGNC-1093,RO:0002206,HGNC HGNC:1093


In [37]:
edges1[edges1['object'].str.contains('&')]

,subject,predicate,object


In [38]:
fulldata['celltype_singleCell_predicate'] = 'RO:0002206'  # 'expressed in'  'cell_type_has_single_cell_expression'

# cell type to singe cell
edges2 = fulldata[['CODEID','celltype_singleCell_predicate','celltype_CODEID']]
edges2.columns = ['subject','predicate','object']

In [39]:
edges2

,subject,predicate,object
0,SCHEART Erythrocytes-HGNC-1093,RO:0002206,author_defined_cluster Erythrocytes
1,SCHEART Epicardium-derived-cells-HGNC-1093,RO:0002206,author_defined_cluster Epicardium-derived-cells
2,SCHEART Ventricular-cardiomyocytes-HGNC-1093,RO:0002206,author_defined_cluster Ventricular-cardiomyocytes
3,SCHEART Erythrocytes-HGNC-7400,RO:0002206,author_defined_cluster Erythrocytes
4,SCHEART Erythrocytes-HGNC-4703,RO:0002206,author_defined_cluster Erythrocytes
...,...,...,...
2811,SCHEART Fibroblast-like-(related-to-larger-vas...,RO:0002206,author_defined_cluster Fibroblast-like-(relate...
2812,SCHEART Fibroblast-like-(related-to-larger-vas...,RO:0002206,author_defined_cluster Fibroblast-like-(relate...
2813,SCHEART Fibroblast-like-(related-to-larger-vas...,RO:0002206,author_defined_cluster Fibroblast-like-(relate...
2814,SCHEART Fibroblast-like-(related-to-larger-vas...,RO:0002206,author_defined_cluster Fibroblast-like-(relate...


In [40]:
edges = pd.concat([edges1,edges2])

In [41]:
edges2[edges2['object'].str.contains('&')]

,subject,predicate,object


In [42]:
assert len(edges1) == 2816
assert len(edges2) == 2816

## Scroll down to code to create bins for p-vals and log2fc for single cell expression Terms (past the CUI/CUI-CUI, CODEs files)

## SUIs and CODE-SUIs will both be created in 2 steps.
First create the p-value and log2fc Terms and then create the threshold Terms

### Create bins for p-values and log2fc
I  took the bins list straight from gtex, so we have the exact same bins for this data.

In [43]:
# Split p-vals up into bins

# Define pval Bins:
#### EXACT SAME BINS from GTEx notebook. ######
bins = [0,1e-12,1e-11,1e-10,1e-9,1e-8,1e-7,1e-6,1e-5,1e-4,1e-3,.005,.01,.02,.03,.04,.05,.06]

# Bin pvals
fulldata['pvalue_bins'] = pd.cut(fulldata['p_val_adj'], bins)

# Now bin the log2FC column

In [44]:
log2fc_bins_neg = [-5,-4,-3,-2.5,-2,-1.75,-1.5,-1.25,-1,-.75,-.5,-.25,-.2,-.15,-.1,-.05]
log2fc_bins_pos = [i*-1 for i in log2fc_bins_neg][::-1] # and reverse it.
log2fc_bins =  log2fc_bins_neg + [0] + log2fc_bins_pos + [6,7]
fulldata['log2fc_bins'] = pd.cut(fulldata['avg_log2FC'], log2fc_bins)

In [45]:
# pvalues that are 0 need to be addded to the (0.0,1e-12] bin. The lowerbound for this bin is not inclusive so
# 0's are not automatically added to it.

fulldata['pval_bins'] = [i if i is not np.nan else '(0.0,1e-12]' for i in fulldata['pvalue_bins']]
fulldata.drop('pvalue_bins',axis=1,inplace=True)

In [46]:
# Remove [] and () characters from intervals before creating SUIs
fulldata['log2fc_bins'] = [str(i)[1:-1] for i in fulldata['log2fc_bins']]
fulldata['pval_bins'] = [str(i)[1:-1] for i in fulldata['pval_bins']]

fulldata['log2fc_bins'] = fulldata['log2fc_bins'].str.replace(' ','')
fulldata['pval_bins'] = fulldata['pval_bins'].str.replace(' ','')

In [47]:
assert fulldata.isna().sum().sum() == 0
assert len(fulldata[fulldata['log2fc_bins'].isna()]) == 0

In [48]:
fulldata

,avg_log2FC,p_val_adj,cluster,gene,cluster_celltype_name,hgnc_id,celltype_CODE,hgnc_id_dashes,SAB,CODE,CODEID,celltype_SAB,celltype_CODEID,hgnc_codeID,hgnc_singleCell_predicate,celltype_singleCell_predicate,log2fc_bins,pval_bins
0,5.513025,0.000000e+00,7,BPGM,Erythrocytes,HGNC:1093,Erythrocytes,HGNC-1093,SCHEART,Erythrocytes-HGNC-1093,SCHEART Erythrocytes-HGNC-1093,author_defined_cluster,author_defined_cluster Erythrocytes,HGNC HGNC:1093,RO:0002206,RO:0002206,"5.0,6.0","0.0,1e-12"
1,-2.470632,1.081173e-02,6,BPGM,Epicardium-derived cells,HGNC:1093,Epicardium-derived-cells,HGNC-1093,SCHEART,Epicardium-derived-cells-HGNC-1093,SCHEART Epicardium-derived-cells-HGNC-1093,author_defined_cluster,author_defined_cluster Epicardium-derived-cells,HGNC HGNC:1093,RO:0002206,RO:0002206,"-2.5,-2.0","0.01,0.02"
2,-1.961892,1.570796e-05,14,BPGM,Ventricular cardiomyocytes,HGNC:1093,Ventricular-cardiomyocytes,HGNC-1093,SCHEART,Ventricular-cardiomyocytes-HGNC-1093,SCHEART Ventricular-cardiomyocytes-HGNC-1093,author_defined_cluster,author_defined_cluster Ventricular-cardiomyocytes,HGNC HGNC:1093,RO:0002206,RO:0002206,"-2.0,-1.75","1e-05,0.0001"
3,5.509262,0.000000e+00,7,MT1H,Erythrocytes,HGNC:7400,Erythrocytes,HGNC-7400,SCHEART,Erythrocytes-HGNC-7400,SCHEART Erythrocytes-HGNC-7400,author_defined_cluster,author_defined_cluster Erythrocytes,HGNC HGNC:7400,RO:0002206,RO:0002206,"5.0,6.0","0.0,1e-12"
4,4.710317,0.000000e+00,7,GYPB,Erythrocytes,HGNC:4703,Erythrocytes,HGNC-4703,SCHEART,Erythrocytes-HGNC-4703,SCHEART Erythrocytes-HGNC-4703,author_defined_cluster,author_defined_cluster Erythrocytes,HGNC HGNC:4703,RO:0002206,RO:0002206,"4.0,5.0","0.0,1e-12"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2811,1.037628,1.397046e-43,9,EPS8,Fibroblast-like (related to larger vascular de...,HGNC:3420,Fibroblast-like-(related-to-larger-vascular-de...,HGNC-3420,SCHEART,Fibroblast-like-(related-to-larger-vascular-de...,SCHEART Fibroblast-like-(related-to-larger-vas...,author_defined_cluster,author_defined_cluster Fibroblast-like-(relate...,HGNC HGNC:3420,RO:0002206,RO:0002206,"1.0,1.25","0.0,1e-12"
2812,1.006935,5.950515e-42,9,SEMA5A,Fibroblast-like (related to larger vascular de...,HGNC:10736,Fibroblast-like-(related-to-larger-vascular-de...,HGNC-10736,SCHEART,Fibroblast-like-(related-to-larger-vascular-de...,SCHEART Fibroblast-like-(related-to-larger-vas...,author_defined_cluster,author_defined_cluster Fibroblast-like-(relate...,HGNC HGNC:10736,RO:0002206,RO:0002206,"1.0,1.25","0.0,1e-12"
2813,1.072414,8.713596e-40,9,MFGE8,Fibroblast-like (related to larger vascular de...,HGNC:7036,Fibroblast-like-(related-to-larger-vascular-de...,HGNC-7036,SCHEART,Fibroblast-like-(related-to-larger-vascular-de...,SCHEART Fibroblast-like-(related-to-larger-vas...,author_defined_cluster,author_defined_cluster Fibroblast-like-(relate...,HGNC HGNC:7036,RO:0002206,RO:0002206,"1.0,1.25","0.0,1e-12"
2814,1.082802,1.299664e-38,9,ZEB2,Fibroblast-like (related to larger vascular de...,HGNC:14881,Fibroblast-like-(related-to-larger-vascular-de...,HGNC-14881,SCHEART,Fibroblast-like-(related-to-larger-vascular-de...,SCHEART Fibroblast-like-(related-to-larger-vas...,author_defined_cluster,author_defined_cluster Fibroblast-like-(relate...,HGNC HGNC:14881,RO:0002206,RO:0002206,"1.0,1.25","0.0,1e-12"


In [49]:
fulldata['celltype_label'] = np.nan
fulldata['singlecell_label'] = np.nan

# Now create nodes file
Need to create a 'single-cell' nodes file and a 'cell type' nodes file

In [50]:
# cell type nodes file
nodes_celltypes = fulldata[['celltype_CODEID','celltype_label']]

# Celltype nodes dont have log2fc or pvals so create nan cols so we can concatenate to the singlecell nodes file
#nodes_celltypes['log2fc_bins'] = np.nan
#nodes_celltypes['pval_bins'] = np.nan

# single cell nodes file
nodes_singlecell = fulldata[['CODEID','singlecell_label']] #

# log2fc_bins pval_bins ---> additional columns for upper and lower threshold? (4 float columns total)

nodes_celltypes.columns = ['node_id','node_label']
nodes_singlecell.columns = ['node_id','node_label']

nodes = pd.concat([nodes_celltypes,nodes_singlecell])

## Create log2FC edge file

In [51]:
fulldata['CODEID_log2FC'] = ['LOG2FCBINS ' + i for i in fulldata['log2fc_bins']]
edges_scHeart_log2FC = fulldata[['CODEID','CODEID_log2FC']]

In [52]:
edges_scHeart_log2FC['predicate'] = 'log2FC'

In [53]:
edges_scHeart_log2FC = edges_scHeart_log2FC[['CODEID','predicate','CODEID_log2FC']]
edges_scHeart_log2FC.columns = ['subject','predicate','object']


In [54]:
edges = pd.concat([edges,edges_scHeart_log2FC])

In [55]:
edges.sample(10)

,subject,predicate,object
1016,SCHEART Fibroblast-like-(related-to-smaller-va...,RO:0002206,HGNC HGNC:9630
2635,SCHEART Cardiac-neural-crest-cells-and-Schwann...,RO:0002206,HGNC HGNC:1704
507,SCHEART Fibroblast-like-(related-to-larger-vas...,RO:0002206,author_defined_cluster Fibroblast-like-(relate...
205,SCHEART Erythrocytes-HGNC-695,RO:0002206,author_defined_cluster Erythrocytes
2640,SCHEART Cardiac-neural-crest-cells-and-Schwann...,log2FC,"LOG2FCBINS 1.0,1.25"
2427,SCHEART Capillary-endothelium-HGNC-3176,RO:0002206,author_defined_cluster Capillary-endothelium
2320,SCHEART Immune-cells-HGNC-1765,log2FC,"LOG2FCBINS 1.0,1.25"
2425,SCHEART Capillary-endothelium-HGNC-1759,RO:0002206,HGNC HGNC:1759
1172,SCHEART Erythrocytes-HGNC-21689,RO:0002206,HGNC HGNC:21689
2604,SCHEART Cardiac-neural-crest-cells-and-Schwann...,log2FC,"LOG2FCBINS 2.0,2.5"


# Save edges

In [58]:
edges.to_csv('/Users/stearb/Desktop/DESKTOP_TRANSFER/DataDistilleryFiles/scHeart/OWLNETS_edgelist.txt',
             sep='\t',index=False)

## need nodes for avg logFC but NOT for pval (GTEx ingest includes P_VALUE_BINS)

In [59]:
nodes_log2fc = pd.DataFrame(edges_scHeart_log2FC['object'].drop_duplicates())

In [60]:
nodes_log2fc['node_label'] = np.nan

In [61]:
nodes_log2fc.columns = ['node_id','node_label']
nodes_log2fc.head(3)

,node_id,node_label
0,"LOG2FCBINS 5.0,6.0",NaN
1,"LOG2FCBINS -2.5,-2.0",NaN
2,"LOG2FCBINS -2.0,-1.75",NaN


In [62]:
nodes = pd.concat([nodes,nodes_log2fc])

nodes = nodes.drop_duplicates(subset=['node_id'])

In [63]:
dict(Counter([i[0] for i in nodes['node_id'].str.split(' ')]))

{'author_defined_cluster': 14, 'SCHEART': 2816, 'LOG2FCBINS': 17}

In [64]:
# other colls can be nan
nodes['node_synonyms'] = np.nan
nodes['node_namespace'] = np.nan
nodes['node_dbxrefs'] = np.nan
nodes['node_definition'] = np.nan

assert len(nodes) == 2847

In [66]:
def fill_missing_cols(df):
    
    if 'node_id' not in df.columns:
        raise ValueError('Must have at least a "node_id" column.')
        
    all_cols = set([ 'node_label', 'node_synonyms', 'node_dbxrefs',
            'node_definition','node_namespace','value','lowerbound','upperbound','unit'])
   
    missing_cols = list(all_cols - set(df.columns))
    nan_cols_df = pd.DataFrame(np.full([len(df), len(missing_cols)], np.nan),columns=missing_cols)
    nan_cols_df.index = df.index
    return pd.concat([df,nan_cols_df],axis=1)

In [68]:
nodes = fill_missing_cols(nodes)

In [70]:
nodes.to_csv('/Users/stearb/Desktop/DESKTOP_TRANSFER/DataDistilleryFiles/scHeart/OWLNETS_node_metadata.txt',
             sep='\t',index=False)

In [57]:
a=set(edges[edges['object'].str.startswith('scHeart_PMID_31835037 ')]['object'].drop_duplicates())

In [58]:
b=set(edges[edges['subject'].str.startswith('scHeart_PMID_31835037 ')]['subject'])

In [66]:
c=set(nodes[nodes['node_id'].str.startswith('scHeart_PMID_31835037')]['node_id'])

In [67]:
a == b == c

True

In [50]:
# SABs: author_defiined_cluster, scHeart_PMID_31835037, LOG2FC_BINS

In [56]:
pd.options.display.max_colwidth = 100

In [ ]:
# Cypher checks

# match (cc:Code {SAB:'scHeart_PMID_31835037'})   return count(distinct cc)    # 2816
# match (cc:Code {SAB:'author_defined_cluster'})   return count(distinct cc)  # 14
# match (cc:Code {SAB:'LOG2FC_BINS'})   return cc.CODE       # 17

In [55]:
celltypes_in_kg = ["Erythrocytes",
"Epicardium-derived-cells",
"Ventricular-cardiomyocytes",
"Fibroblast-like-(related-to-cardiac-skeleton-connective-tissue)",
"Capillary-endothelium",
"Immune-cells",
"Fibroblast-like-(related-to-smaller-vascular-development)",
"Cardiac-neural-crest-cells-&-Schwann-progenitor-cells",
"Fibroblast-like-(related-to-larger-vascular-development)",
"Epicardial-cells",
"Myoz2-enriched-cardiomyocytes",
"Atrial-cardiomyocytes"]

In [65]:
all_cts = [i[1] for i in nodes[nodes['node_id'].str.startswith('author-')]['node_id'].str.split(' ')]

In [19]:
#set(all_cts) - set(celltypes_in_kg)

In [75]:
len(np.unique([i[1] for i  in edges[edges['subject'].str.startswith('author-')]['subject'].str.split(' ')]))

14

## Create pval edges -- still need to complete

In [68]:
'''
fulldata['pval_bins_CODEID'] = ['P_VALUE_BINS '+i for i in fulldata['pval_bins']]

edges_scHeart_pvals = fulldata[['CODEID','pval_bins_CODEID']]
edges_scHeart_pvals['predicate'] = 'p_value'
edges_scHeart_pvals = edges_scHeart_pvals[['CODEID','predicate','pval_bins_CODEID']]
edges_scHeart_pvals.columns = ['subject','predicate','object']
edges_scHeart_pvals'''

,subject,predicate,object
0,scHeart_PMID:31835037 Erythrocytes_HGNC:1093,p_value,"P_VALUE_BINS 0.0,1e-12"
1,scHeart_PMID:31835037 Epicardium-derived_cells...,p_value,"P_VALUE_BINS 0.01,0.02"
2,scHeart_PMID:31835037 Ventricular_cardiomyocyt...,p_value,"P_VALUE_BINS 1e-05,0.0001"
3,scHeart_PMID:31835037 Erythrocytes_HGNC:7400,p_value,"P_VALUE_BINS 0.0,1e-12"
4,scHeart_PMID:31835037 Erythrocytes_HGNC:4703,p_value,"P_VALUE_BINS 0.0,1e-12"
...,...,...,...
2811,scHeart_PMID:31835037 Fibroblast-like_(related...,p_value,"P_VALUE_BINS 0.0,1e-12"
2812,scHeart_PMID:31835037 Fibroblast-like_(related...,p_value,"P_VALUE_BINS 0.0,1e-12"
2813,scHeart_PMID:31835037 Fibroblast-like_(related...,p_value,"P_VALUE_BINS 0.0,1e-12"
2814,scHeart_PMID:31835037 Fibroblast-like_(related...,p_value,"P_VALUE_BINS 0.0,1e-12"


In [54]:
nodes[nodes['node_id'].str.contains('Atrial-cardiomyocytes-HGNC-10446')]

,node_id,node_label,node_synonyms,node_namespace,node_dbxrefs,node_definition
2495,scHeart-PMID-31835037 Atrial-cardiomyocytes-HG...,NaN,NaN,NaN,NaN,NaN


# Create the Threshold Terms (they will be strings)

Thresholds:
- < .05
- < .001
- < .0001
- < 1e-10

### Assign Threshold values 

In [78]:
fulldata['pval_threshold_1e-10'] = ['< 1e-10' if i < 1e-10 else np.nan for i in fulldata['p_val_adj']]

fulldata['pval_threshold_1e-4'] = ['< 0.0001' if i < 1e-4 else np.nan for i in fulldata['p_val_adj']]

fulldata['pval_threshold_1e-3'] = ['< 0.001' if i < 1e-3 else np.nan for i in fulldata['p_val_adj']]

fulldata['pval_threshold_1e-2'] = ['< 0.01' if i < 1e-3 else np.nan for i in fulldata['p_val_adj']]

fulldata['pval_threshold_.05'] = ['< 0.05' if i < .05 else np.nan for i in fulldata['p_val_adj']]


thresh_df = fulldata[['pval_threshold_1e-10','pval_threshold_1e-4',
                  'pval_threshold_1e-3','pval_threshold_1e-2','pval_threshold_.05','p_val_adj']]

thresh_df.head(3)

,pval_threshold_1e-10,pval_threshold_1e-4,pval_threshold_1e-3,pval_threshold_1e-2,pval_threshold_.05,p_val_adj
0,< 1e-10,< 0.0001,< 0.001,< 0.01,< 0.05,0.000000
1,NaN,NaN,NaN,NaN,< 0.05,0.010812
2,NaN,< 0.0001,< 0.001,< 0.01,< 0.05,0.000016


In [81]:
pd.set_option('display.max_columns', None)